In [18]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append(os.path.abspath(os.path.join("..")))

import pandas as pd

from icu_pipeline.pipeline import (
    Pipeline,
    DataSource,
    SourceConfig,
)

PG_PASS = os.getenv("PG_PASS")
PG_USER = os.getenv("PG_USER")
PG_HOST = os.getenv("PG_HOST")
PG_PORT = os.getenv("PG_PORT")

MIMIC_DB = os.getenv("PG_MIMIC_NAME")
EICU_DB = os.getenv("PG_EICU_NAME")
AMDS_DB = os.getenv("PG_AMDS_NAME")
SIC_DB = os.getenv("PG_SIC_NAME")
HIRID_DB = os.getenv("PG_HIRID_NAME")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
mimic_config = SourceConfig(
    f"postgresql+psycopg://{PG_USER}:{PG_PASS}@{PG_HOST}/{MIMIC_DB}",
    limit=10,
    chunksize=10
)
eicu_config = SourceConfig(
    f"postgresql+psycopg://{PG_USER}:{PG_PASS}@{PG_HOST}/{EICU_DB}",
    limit=10,
    chunksize=10
)
amds_config = SourceConfig(
    f"postgresql+psycopg://{PG_USER}:{PG_PASS}@{PG_HOST}/{AMDS_DB}",
    limit=10,
    chunksize=10
)
sic_config = SourceConfig(
    f"postgresql+psycopg://{PG_USER}:{PG_PASS}@{PG_HOST}/{SIC_DB}",
    limit=10,
    chunksize=10
)
hirid_config = SourceConfig(
    f"postgresql+psycopg://{PG_USER}:{PG_PASS}@{PG_HOST}/{HIRID_DB}",
    limit=10,
    chunksize=10
)

In [20]:
from icu_pipeline.sink.pandas import PandasSink

pipeline = Pipeline(
    source_configs={
        #DataSource.MIMICIV: mimic_config,
        DataSource.EICU: eicu_config,
    },
    sink_mapper=PandasSink(),
)

concepts = ["HeartRate"]

In [21]:
from pathlib import Path

base_path = str(Path.cwd().parent / "conceptbase" / "concepts")

result = pipeline.transform(concepts, base_path)
print(pipeline._graph)

Pipe Class: InMemoryPipe
Node Class: InMemoryNode


In [22]:
result = [{
    k: v for k,v in c.items()
} for c in result]
out = {}
for c in pipeline._sink_mapper._sources:
    out[c] = pd.concat([v[c] for v in result])
out["HeartRate"]

DEBUG:root:Added 0 new dependencies to the Graph.


DEBUG:root:
            SELECT DISTINCT "patienthealthsystemstayid"
            FROM "eicu_crd"."patient"
            LIMIT 10
        
DEBUG:root:
                SELECT "heartrate" AS "value", "patienthealthsystemstayid" AS "patient_id", "hospitaladmittime24" AS "time", "hospitaldischargeyear" AS "year", "observationoffset" AS "offset"
                FROM "eicu_crd"."vitalperiodic"
                INNER JOIN "eicu_crd"."patient" ON "eicu_crd"."vitalperiodic"."patientunitstayid" = "eicu_crd"."patient"."patientunitstayid"
                WHERE "heartrate" IS NOT NULL
                AND patienthealthsystemstayid IN (128919,128927,128941,128943,128948,128952,128968,128970,128973,128982)
            


Pipe Class: InMemoryPipe
Node Class: InMemoryNode
Concept(5,HeartRate)-->FrequencyConverter(6)-->PandasSink(4)


,subject,effective_date_time,value_quantity,code
0,"{'reference': '128919', 'type': 'eicu'}",2015-01-03 02:13:00+00:00,"{'value': 92.0, 'unit': 'bpm'}","{'coding': {'code': 'HeartRate', 'system': 'sn..."
1,"{'reference': '128919', 'type': 'eicu'}",2015-01-02 13:23:00+00:00,"{'value': 118.0, 'unit': 'bpm'}","{'coding': {'code': 'HeartRate', 'system': 'sn..."
2,"{'reference': '128919', 'type': 'eicu'}",2015-01-02 21:48:00+00:00,"{'value': 78.0, 'unit': 'bpm'}","{'coding': {'code': 'HeartRate', 'system': 'sn..."
3,"{'reference': '128919', 'type': 'eicu'}",2015-01-02 14:48:00+00:00,"{'value': 118.0, 'unit': 'bpm'}","{'coding': {'code': 'HeartRate', 'system': 'sn..."
4,"{'reference': '128919', 'type': 'eicu'}",2015-01-03 01:53:00+00:00,"{'value': 92.0, 'unit': 'bpm'}","{'coding': {'code': 'HeartRate', 'system': 'sn..."
...,...,...,...,...
6814,"{'reference': '128982', 'type': 'eicu'}",2015-01-02 15:21:00+00:00,"{'value': 104.0, 'unit': 'bpm'}","{'coding': {'code': 'HeartRate', 'system': 'sn..."
6815,"{'reference': '128982', 'type': 'eicu'}",2015-01-01 13:41:00+00:00,"{'value': 98.0, 'unit': 'bpm'}","{'coding': {'code': 'HeartRate', 'system': 'sn..."
6816,"{'reference': '128982', 'type': 'eicu'}",2015-01-03 15:31:00+00:00,"{'value': 96.0, 'unit': 'bpm'}","{'coding': {'code': 'HeartRate', 'system': 'sn..."
6817,"{'reference': '128982', 'type': 'eicu'}",2015-01-01 15:06:00+00:00,"{'value': 100.0, 'unit': 'bpm'}","{'coding': {'code': 'HeartRate', 'system': 'sn..."


In [23]:
out["HeartRate"]

,subject,effective_date_time,value_quantity,code
0,"{'reference': '128919', 'type': 'eicu'}",2015-01-03 02:13:00+00:00,"{'value': 92.0, 'unit': 'bpm'}","{'coding': {'code': 'HeartRate', 'system': 'sn..."
1,"{'reference': '128919', 'type': 'eicu'}",2015-01-02 13:23:00+00:00,"{'value': 118.0, 'unit': 'bpm'}","{'coding': {'code': 'HeartRate', 'system': 'sn..."
2,"{'reference': '128919', 'type': 'eicu'}",2015-01-02 21:48:00+00:00,"{'value': 78.0, 'unit': 'bpm'}","{'coding': {'code': 'HeartRate', 'system': 'sn..."
3,"{'reference': '128919', 'type': 'eicu'}",2015-01-02 14:48:00+00:00,"{'value': 118.0, 'unit': 'bpm'}","{'coding': {'code': 'HeartRate', 'system': 'sn..."
4,"{'reference': '128919', 'type': 'eicu'}",2015-01-03 01:53:00+00:00,"{'value': 92.0, 'unit': 'bpm'}","{'coding': {'code': 'HeartRate', 'system': 'sn..."
...,...,...,...,...
6814,"{'reference': '128982', 'type': 'eicu'}",2015-01-02 15:21:00+00:00,"{'value': 104.0, 'unit': 'bpm'}","{'coding': {'code': 'HeartRate', 'system': 'sn..."
6815,"{'reference': '128982', 'type': 'eicu'}",2015-01-01 13:41:00+00:00,"{'value': 98.0, 'unit': 'bpm'}","{'coding': {'code': 'HeartRate', 'system': 'sn..."
6816,"{'reference': '128982', 'type': 'eicu'}",2015-01-03 15:31:00+00:00,"{'value': 96.0, 'unit': 'bpm'}","{'coding': {'code': 'HeartRate', 'system': 'sn..."
6817,"{'reference': '128982', 'type': 'eicu'}",2015-01-01 15:06:00+00:00,"{'value': 100.0, 'unit': 'bpm'}","{'coding': {'code': 'HeartRate', 'system': 'sn..."


In [6]:
out["HeartRate"].iloc[0]

subject                          {'reference': '128919', 'type': 'eicu'}
effective_date_time                            2015-01-03 02:13:00+00:00
value_quantity                            {'value': 92.0, 'unit': 'bpm'}
code                   {'coding': {'code': 'HeartRate', 'system': 'sn...
Name: 0, dtype: object

In [7]:
print(pipeline._graph)

Pipe Class: InMemoryPipe
Node Class: InMemoryNode
Concept(1,HeartRate)-->FrequencyConverter(2)-->PandasSink(0)


In [8]:
pipeline._sink_mapper._sources

{'HeartRate': <icu_pipeline.graph.in_memory.InMemoryPipe at 0x7f9c1e5b3530>}